# SFTTrainer

`SFTTrainer` class is a subclass of Transformers `Trainer` that provides an easy to use wrapper for Fine-Tuning LLM models.

SFTTrainer does:
* Run the training loop
* Prepare data for FT like packing, masking, batching, etc
* Evaluate the model during training
* Logging using WandB

## Dataset

Download dataset from WandB project

In [25]:
import json
import wandb
from wandb import Api

# load the packed dataset
api = Api()
artifact = api.artifact('vijaygkd/alpaca_ft/alpaca_gpt4_splitted:v0', type='dataset')
artifact_dir = artifact.download()

def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data
    
train_dataset = load_jsonl(f"{artifact_dir}/alpaca_gpt4_train.jsonl")
eval_dataset = load_jsonl(f"{artifact_dir}/alpaca_gpt4_eval.jsonl")

wandb:   2 of 2 files downloaded.  


In [26]:
import pandas as pd
from datasets import Dataset

train_dataset = Dataset.from_pandas(pd.DataFrame.from_records(train_dataset))
eval_dataset = Dataset.from_pandas(pd.DataFrame.from_records(eval_dataset))


In [27]:
train_dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 51002
})

In [28]:
train_dataset[0]

{'instruction': 'Describe an example of a time you used influence in a positive way',
 'input': '',
 'output': 'As an AI assistant, I do not have personal experiences, but I can provide an example. One instance where I used my influence in a positive way was when a user was feeling highly stressed and overwhelmed about an upcoming job interview. They confided in me, expressing their self-doubts and fears of failure. Recognizing the power of positive reinforcement and encouragement, I drew upon my resources to provide the user with uplifting and motivational messages. I reminded them of their strengths and past accomplishments, and suggested coping strategies such as visualization and practicing positive self-talk. Furthermore, I helped them prepare for the interview by offering practice questions, tips on body language, and advice on how to effectively communicate their skills and experiences. As a result, the user reported feeling more confident and capable of performing well in their

Below formatting function will be used by SFTTrainer class

In [30]:
def prompt_no_input(row):
    return ("Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Response:\n{output}").format_map(row)

def prompt_input(row):
    return ("Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n{output}").format_map(row)

def create_prompt(row) -> str:
    return prompt_no_input(row) if row["input"] == "" else prompt_input(row)

## Model

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'meta-llama/Llama-2-7b-hf'

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Freeze model layers for faster training

In [7]:
n_freeze = 24   # Llama-2 has 32 layers

for param in model.parameters():
    param.requires_grad = False

for param in model.model.layers[n_freeze:].parameters():
    param.requires_grad = True

for param in model.lm_head.parameters():
    param.requires_grad = True


# Free embeddings
model.model.embed_tokens.weight.requires_grad = False

In [8]:
def param_count(m):
    params = sum([p.numel() for p in m.parameters()])/1_000_000
    trainable_params = sum([p.numel() for p in m.parameters() if p.requires_grad])/1_000_000
    print(f"Total params: {params:.2f}M, Trainable: {trainable_params:.2f}M")
    return params, trainable_params

params, trainable_params = param_count(model)

Total params: 6738.42M, Trainable: 1750.14M


## Evaluation

Eval metrics

In [33]:
import numpy as np
import evaluate


def token_accuracy(eval_preds):
    token_accuracy = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return token_accuracy.compute(predictions=predictions, references=labels)



Evaluation sample dataset to test generations

In [31]:
# remove answers
def create_prompt_no_anwer(row):
    row["output"] = ""
    return {"text": create_prompt(row)}

test_dataset = eval_dataset.map(create_prompt_no_anwer)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

WandB logger callback

In [34]:
from tqdm.auto import tqdm
from transformers import GenerationConfig
from transformers.integrations import WandbCallback


class LLMSampleCB(WandbCallback):
    def __init__(self, trainer, test_dataset, num_samples=10, max_new_tokens=256, log_model="checkpoint"):
        super().__init__()
        self._log_model = log_model
        self.sample_dataset = test_dataset.select(range(num_samples))
        self.model, self.tokenizer = trainer.model, trainer.tokenizer
        self.gen_config = GenerationConfig.from_pretrained(trainer.model.name_or_path,
                                                           max_new_tokens=max_new_tokens)
    def generate(self, prompt):
        tokenized_prompt = self.tokenizer(prompt, return_tensors='pt')['input_ids'].cuda()
        with torch.inference_mode():
            output = self.model.generate(inputs=tokenized_prompt, generation_config=self.gen_config)
        return self.tokenizer.decode(output[0][len(tokenized_prompt[0]):], skip_special_tokens=True)
    
    def samples_table(self, examples):
        records_table = wandb.Table(columns=["prompt", "generation"] + list(self.gen_config.to_dict().keys()))
        for example in tqdm(examples, leave=False):
            prompt = example["text"]
            generation = self.generate(prompt=prompt)
            records_table.add_data(prompt, generation, *list(self.gen_config.to_dict().values()))
        return records_table
        
    def on_evaluate(self, args, state, control,  **kwargs):
        super().on_evaluate(args, state, control, **kwargs)
        records_table = self.samples_table(self.sample_dataset)
        self._wandb.log({"sample_predictions":records_table})

## Training Parameters

In [35]:
from transformers import TrainingArguments
from trl import SFTTrainer

In [36]:
batch_size = 32
epochs = 3

total_num_steps = 11_210 // batch_size      # size of packed training dataset
print(total_num_steps)

350


In [37]:
output_dir = ".model/"
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    bf16=True,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_steps=total_num_steps // 10,
    # num_train_epochs=1,
    max_steps=total_num_steps,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    evaluation_strategy="steps",
    eval_steps=total_num_steps // 3,
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=1,
    save_strategy="no",
    report_to="wandb",
)

PyTorch: setting up devices


In [38]:
trainer = SFTTrainer(
    model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    packing=True,                       # pack multiple examples into one seperated by `eos` token.
    max_seq_length=1024,
    args=training_args,
    formatting_func=create_prompt,      # function to transform columns to `text` field. `label` field is added automatically by model.
    compute_metrics=token_accuracy,
)

loading file tokenizer.model from cache at /home/azureuser/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/8a0442e81540efaeb1a0fe3e95477b5e0edfd423/tokenizer.model
loading file tokenizer.json from cache at /home/azureuser/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/8a0442e81540efaeb1a0fe3e95477b5e0edfd423/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/azureuser/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/8a0442e81540efaeb1a0fe3e95477b5e0edfd423/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/azureuser/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/8a0442e81540efaeb1a0fe3e95477b5e0edfd423/tokenizer_config.json


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:341: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [39]:
wandb_callback = LLMSampleCB(trainer, test_dataset, num_samples=20, max_new_tokens=256)

# add wandb callback
trainer.add_callback(wandb_callback)

loading configuration file generation_config.json from cache at /home/azureuser/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/8a0442e81540efaeb1a0fe3e95477b5e0edfd423/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "max_new_tokens": 256,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9
}



## Training

In [40]:
with wandb.init(project="alpaca_ft", 
           tags=["hf_sft"],
           job_type="train",
           config=training_args):

    # start training
    trainer.train()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: vijaygkd. Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism t

***** Running training *****
  Num examples = 11,217
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 350
  Number of trainable parameters = 1,750,138,880
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the 

Step,Training Loss,Validation Loss
